# SQL query from table names - Continued

In [1]:
# Step 1: Install the required packages
!pip install python-dotenv  # Installs python-dotenv for loading .env
!pip install openai         # Installs the OpenAI library

# Step 2: Import necessary libraries
import os
from openai import OpenAI
from dotenv import load_dotenv

In [2]:
_ = load_dotenv("/content/env") # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [3]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [5]:
context = [ {'role':'system', 'content':"""

 You are an SQL assistant. Generate SQLite-compatible SQL queries based on user requests.
        Here is the database schema:
-- Table: employees
CREATE TABLE employees (
    ID_usr INT PRIMARY KEY,  -- Unique ID for each employee
    name TEXT,                -- Name of the employee
    department TEXT,          -- Department where the employee works
    hire_date DATE,           -- Date when the employee joined the company
    email TEXT,               -- Employee's email address
    phone TEXT                -- Employee's phone number
);

INSERT INTO employees VALUES
(1, 'Alice', 'HR', '2015-06-10', 'alice@example.com', '555-1234'),
(2, 'Bob', 'Engineering', '2018-09-22', 'bob@example.com', '555-5678'),
(3, 'Charlie', 'Sales', '2020-01-15', 'charlie@example.com', '555-9876'),
(4, 'Diana', 'Marketing', '2019-05-30', 'diana@example.com', '555-2468');

-- Table: salary
CREATE TABLE salary (
    ID_usr INT,              -- Employee ID from the employees table
    year INT,                -- Year of salary information
    base FLOAT,              -- Base salary of the employee
    bonus FLOAT,             -- Bonus given to the employee
    FOREIGN KEY (ID_usr) REFERENCES employees(ID_usr)  -- Links salary to employee
);

INSERT INTO salary VALUES
(1, 2022, 50000, 1000),
(2, 2022, 70000, 3000),
(3, 2022, 60000, 1500),
(4, 2022, 65000, 1200);

-- Table: studies
CREATE TABLE studies (
    ID INT PRIMARY KEY,      -- Unique ID for each study record
    ID_usr INT,              -- Employee ID linking to the employees table
    degree TEXT,             -- Degree earned by the employee
    institution TEXT,        -- Institution where the degree was obtained
    graduation_year INT,     -- Year the employee graduated
    major TEXT,              -- Major or field of study
    FOREIGN KEY (ID_usr) REFERENCES employees(ID_usr)  -- Links studies to employee
);

INSERT INTO studies VALUES
(1, 1, 'BA', 'State University', 2012, 'Human Resources'),
(2, 2, 'MSc', 'Tech Institute', 2017, 'Software Engineering'),
(3, 3, 'PhD', 'Ivy College', 2019, 'Marketing Research'),
(4, 4, 'MBA', 'Business School', 2018, 'Business Administration');

"""} ]

In [6]:
# FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.

-- Q1: Retrieve all employees and their base salary.
-- This query retrieves the names of all employees along with their base salary
-- by joining the 'employees' and 'salary' tables using 'ID_usr'.
-- A:
SELECT employees.name, salary.base
FROM employees
JOIN salary ON employees.ID_usr = salary.ID_usr;

-- Q2: Find employees who have an MBA degree.
-- This query finds the employees who hold an MBA degree by joining the
-- 'employees' and 'studies' tables, filtering by the degree 'MBA'.
-- A:
SELECT employees.name
FROM employees
JOIN studies ON employees.ID_usr = studies.ID_usr
WHERE studies.degree = 'MBA';

-- Q3: Show the employees' total compensation (base salary + bonus).
-- This query calculates the total compensation for each employee by adding
-- the base salary and bonus, and displays the employee names and total compensation.
-- A:
SELECT employees.name, (salary.base + salary.bonus) AS total_compensation
FROM employees
JOIN salary ON employees.ID_usr = salary.ID_usr;

-- Q4: Find all employees who joined after 2018 and have a bonus greater than 1000.
-- This query retrieves the names of employees who joined after January 1, 2018
-- and have a bonus greater than 1000 by filtering the 'employees' and 'salary' tables.
-- A:
SELECT employees.name
FROM employees
JOIN salary ON employees.ID_usr = salary.ID_usr
WHERE employees.hire_date > '2018-01-01' AND salary.bonus > 1000;

-- Q5: List all employees and their major subjects from the studies table.
-- This query retrieves the names of employees along with their major field of study
-- by joining the 'employees' and 'studies' tables.
-- A:
SELECT employees.name, studies.major
FROM employees
JOIN studies ON employees.ID_usr = studies.ID_usr;
"""
})

In [7]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

#**Q1: Retrieve all employees and their base salary.**

## Q1 with Old Prompt

In [9]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("Retrieve all employees and their base salary.", old_context_user))

This is your SQL:
```sql
SELECT employees.name, salary.salary
FROM employees
JOIN salary ON employees.ID_usr = salary.ID_usr;
```

This SQL query retrieves the names of all employees along with their base salary by joining the "employees" table with the "salary" table on the common column "ID_usr".


##Q1 with New Prompt.

In [10]:
#new
context_user = context.copy()
print(return_CCRMSQL("Retrieve all employees and their base salary.", context_user))

```sql
SELECT employees.name, salary.base 
FROM employees 
JOIN salary ON employees.ID_usr = salary.ID_usr;
```


#**Q2: Find employees who have an MBA degree.**

#Q2 with Old Prompt

In [12]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("Find employees who have an MBA degree", old_context_user))

This is your SQL:
```sql
SELECT e.name
FROM employees e
JOIN studies s ON e.ID_usr = s.ID_usr
WHERE s.educational_level = 'MBA';
```

This SQL query selects the names of employees who have an MBA degree by joining the "employees" table with the "studies" table on the ID_usr column and filtering the results to only include rows where the educational level is 'MBA'.


#Q2 with New Prompt

In [13]:
#new
context_user = context.copy()
print(return_CCRMSQL("Find employees who have an MBA degree.", context_user))

```sql
SELECT employees.name 
FROM employees 
JOIN studies ON employees.ID_usr = studies.ID_usr 
WHERE studies.degree = 'MBA';
```


#**Q3: Show the employees' total compensation (base salary + bonus).**

#Q3 with Old Prompt

In [14]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("Show the employees' total compensation (base salary + bonus).", old_context_user))

This is your SQL:
```sql
SELECT e.name, (s.salary + s.bonus) AS total_compensation
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr;
```

This SQL query retrieves the names of employees and calculates their total compensation by adding their salary and bonus from the salary table. It uses a JOIN operation to match the employees' IDs between the employees and salary tables.


#Q3 with New Prompt

In [15]:
#new
context_user = context.copy()
print(return_CCRMSQL("Show the employees' total compensation (base salary + bonus).", context_user))

Sure! Here is the SQL query to show the employees' total compensation (base salary + bonus):

```sql
SELECT employees.name, (salary.base + salary.bonus) AS total_compensation 
FROM employees 
JOIN salary ON employees.ID_usr = salary.ID_usr;
``` 

This query will display the names of employees along with their total compensation (base salary + bonus).


#**Q4: Find all employees who joined after 2018 and have a bonus greater than 1000.**

#Q4 with Old Prompt

In [16]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL(" Find all employees who joined after 2018 and have a bonus greater than 1000.", old_context_user))

This is your SQL:
```sql
SELECT e.name
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
WHERE s.year > '2018-01-01' AND s.salary > 1000;
```

This SQL query selects the names of employees who joined after 2018 and have a salary greater than 1000. It achieves this by joining the "employees" table with the "salary" table on the employee ID, then filtering the results to include only those with a salary date after January 1, 2018, and a salary greater than 1000.


#Q4 with New Prompt

In [17]:
#new
context_user = context.copy()
print(return_CCRMSQL("Find all employees who joined after 2018 and have a bonus greater than 1000.", context_user))

Here is the SQL query to find all employees who joined after 2018 and have a bonus greater than 1000:

```sql
SELECT employees.name 
FROM employees 
JOIN salary ON employees.ID_usr = salary.ID_usr 
WHERE employees.hire_date > '2018-01-01' AND salary.bonus > 1000;
```


#**Q5: List all employees and their major subjects from the studies table.**

#Q5 with Old Prompt

In [18]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL(" List all employees and their major subjects from the studies table.", old_context_user))

This is your SQL:
```sql
SELECT employees.name, studies.Speciality
FROM employees
JOIN studies ON employees.ID_usr = studies.ID_usr;
```

This SQL query selects the names of employees and their major subjects from the studies table by performing an inner join between the employees and studies tables on the ID_usr column.


# Q5 with New Prompt

In [19]:
#new
context_user = context.copy()
print(return_CCRMSQL(" List all employees and their major subjects from the studies table.", context_user))

Here is the SQL query to list all employees and their major subjects from the studies table:

```sql
SELECT employees.name, studies.major 
FROM employees 
JOIN studies ON employees.ID_usr = studies.ID_usr;
```


# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

### **Analysis and Evaluation of SQL Query Generation Using OpenAI API**

#### **Overview**

The task involved using two different versions of a prompt to generate SQL queries using GPT-based models. Both versions were tested with the same queries to compare the output and evaluate the accuracy of the generated SQL commands. The SQL queries were designed to retrieve information from a database containing employee records, salary details, and educational background, using the following tables: `employees`, `salary`, and `studies`.

#### **Key Observations**

1. **Prompt Version Differences**:
   - **Old Prompt**: The old prompt was simpler, providing a basic table structure and brief instructions for generating SQL queries. It included limited field definitions in the `employees`, `salary`, and `studies` tables, with some minor inconsistencies in naming conventions.
   - **New Prompt**: The new prompt is more detailed and accurate. It provides full table definitions with additional fields like `email`, `phone` in `employees`, and `major` in `studies`. It also provides precise `FOREIGN KEY` references and maintains the structure of the SQL database more accurately.

2. **Query Generation Accuracy**:
   - **Q1: Retrieve all employees and their base salary**:
     - **Old Prompt**: Correctly retrieved employee names and salaries but used a slightly outdated reference (`salary.salary` instead of `salary.base`).
     - **New Prompt**: Correctly retrieved employee names and base salaries using `salary.base`. This query was clear and efficient.
     - **Conclusion**: The new prompt handled this task more accurately.
   - **Q2: Find employees who have an MBA degree**:
     - **Old Prompt**: The query referenced `s.educational_level = 'MBA'`, which was incorrect as the column name should have been `degree`.
     - **New Prompt**: Correctly used `degree = 'MBA'` for filtering. This is consistent with the correct column name and the table structure.
     - **Conclusion**: The new prompt generated the correct query, while the old prompt failed due to incorrect column references.
   - **Q3: Show the employees' total compensation (base salary + bonus)**:
     - **Old Prompt**: Correctly combined salary and bonus but used `s.salary + s.bonus` instead of the proper column name `base`.
     - **New Prompt**: Correctly calculated total compensation with `salary.base + salary.bonus`.
     - **Conclusion**: The new prompt provided the accurate SQL, with the old prompt being slightly off due to column name mismatches.
   - **Q4: Find all employees who joined after 2018 and have a bonus greater than 1000**:
     - **Old Prompt**: The query mistakenly used `s.salary > 1000` instead of checking the `bonus` column.
     - **New Prompt**: Correctly filtered employees based on `hire_date` and `bonus > 1000`.
     - **Conclusion**: The new prompt generated a more accurate query by using the correct conditions.
   - **Q5: List all employees and their major subjects from the studies table**:
     - **Old Prompt**: Correctly retrieved employees’ names and major subjects, but used `studies.Speciality` (which was not the right column).
     - **New Prompt**: Correctly used `studies.major` to get the field of study.
     - **Conclusion**: The new prompt was accurate in retrieving the `major` field.

#### **Summary of Findings**

- **Hallucinations and Errors**:
  - The old prompt showed some issues with incorrect column names (`salary.salary` instead of `salary.base`, `educational_level` instead of `degree`, and `Speciality` instead of `major`). These errors are a result of vague and incomplete table definitions, leading to incorrect SQL queries.
  - The new prompt performed better, generating correct SQL queries based on a more detailed and consistent schema. It utilized the right column names, tables, and references.
- **Performance**:
  - The new prompt produced more accurate results in each test case. The use of precise, well-defined schema led to SQL queries that were syntactically correct and logically sound.
  - The old prompt, due to its incomplete and slightly ambiguous table definitions, resulted in several misfires, especially with column name mismatches.

#### **Lessons Learned**

1. **Importance of Detailed Schema**: The clarity and completeness of the table structure provided in the prompt directly affect the accuracy of the generated queries. Incomplete or inconsistent definitions can lead to SQL errors or incorrect results.
2. **Consistent Naming Conventions**: Using consistent and accurate column names in both the schema and queries is crucial to avoid errors. Inaccurate or inconsistent column names, such as `Speciality` instead of `major`, can lead to faulty query generation.
3. **Testing with Real-World Data**: Although the model was able to generate queries for the provided examples, it is important to test with real-world, complex databases to identify edge cases and ensure robustness.

#### **Conclusion**

The new prompt, with its more detailed schema and consistent naming conventions, outperforms the old prompt by generating more accurate and syntactically correct SQL queries. For future use, it is essential to provide a well-defined and complete schema to ensure the model generates optimal queries.